In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

/home/patriciapmr/.pyenv/versions/3.10.6/envs/chatbot_movies/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [4]:
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [7]:
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

In realms of code, where knowledge thrives,
There dwelled a being, LangChain of lives.
A model vast, with layers deep,
A tapestry of words, secrets to keep.

From text to text, it swiftly flows,
Unveiling meaning, as it knows.
A gentle touch, a guiding hand,
It weaves connections, across the land.

With every query, it takes its flight,
Soaring through data, day and night.
It comprehends, it learns, it grows,
A boundless mind, where wonder shows.

It bridges gaps, between the lines,
Revealing insights, hidden designs.
From sentiment to tone, it can discern,
The essence of language, it does yearn.

In chatbots, it dances with grace,
Engaging in dialogue, at a brisk pace.
It translates tongues, with effortless ease,
Connecting worlds, breaking down unease.

But with its power comes a weighty task,
To use its gifts, for good, not ask.
For in the hands of those who lack care,
Its knowledge could become a dangerous snare.

So let us heed its gentle call,
To wield its might, for the benefit 

In [42]:
chroma = Chroma(persist_directory="./raw_data/chroma_db", embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"))

In [43]:
print(chroma)

In [45]:
len(chroma.get()['documents'])

22923

In [46]:
retriever = chroma.as_retriever()

In [47]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f67e0163670>)

In [61]:
query = "Tell me horror movies with zombies"
docs = chroma.similarity_search(query)
docs

[Document(page_content='id: 1043771\nTitle: The Curse of the Screaming Dead\nYear: 1982.0\nDescription: A group of six friends are out in the rural south on a deer hunt when they start hearing strange noises coming from the woods. An investigation of the sounds leads them to an old Confederate graveyard, which suddenly begins sprouting Rebel zombies. Like the living dead in every other movie, these zombies are hungry for human flesh and soon set to chowing down on the hapless hunters.\nMinutes: 91.0\nRating: 2.52\nGenre: Horror\nActors: Richard Ruxton, Jim Ball, Mimi Ishikawa, Mark Redfield, Judy Dixon, Bumb Roberts, Rebecca Bach, Christopher Gummer, Steve Sandkuhler\nTheme: Bloody vampire horror, Gothic and eerie haunting horror, Gory, gruesome, and slasher horror, Horror, the undead and monster classics, Survival horror and zombie carnage, Monsters, aliens, sci-fi and the apocalypse, Extreme gory horror and cannibalism', metadata={'row': 20127, 'source': 'movies_data.csv'}),
 Documen

In [57]:
template = """
You are a helpful AI assistant.
Answer based on the context provided.
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
#Invoke the retrieval chain
response=retrieval_chain.invoke({"input":"Who are Star Wars actors?"})
#Print the answer to the question
print(response["answer"])

Yes, Mandy Morton is in Star Wars (1977). She plays a Rebel Officer.


In [45]:
#